In [1]:
# Load in the python script containing the same code as the load the data notebook
%run loadData.py
# now we can access train, dev, and test
# along with trainSents, devSents testSents

In [2]:
# Imports

import pprint
pp = pprint.PrettyPrinter(indent=4)

import nltk
from nltk.corpus import stopwords

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from string import punctuation  

import re
import pickle
import os

import csv

from collections import defaultdict


In [3]:
# Core functions

classifier = './stanford/classifiers/english.all.3class.distsim.crf.ser.gz'
jar = './stanford/stanford-ner.jar'

sTagger = StanfordNERTagger(classifier,jar)

punct_tokens = set(punctuation)
extra_tokens = set(["what", "where", "how", "when", "who"])

stop_words = set(stopwords.words('english'))

filter_tokens = extra_tokens.union(punct_tokens).union(stop_words)

In [4]:
# Shim function for later clean

def getStanfordTagging(datasetName):
    fnameTrain = './preCompTags/stanfordTaggedTrain.txt'
    fnameDev = './preCompTags/stanfordTaggedDev.txt'
    fnameTest = './preCompTags/stanfordTaggedTest.txt'
    
    theFilePath = ''
    theSents = []
    if (datasetName == 'train'):
        theFilePath = fnameTrain
        theSents = trainSents
    elif (datasetName == 'dev'):
        theFilePath = fnameDev
        theSents = devSents
    elif (datasetName == 'test'):
        theFilePath = fnameTest
        theSents = testSents
    else :
        raise ValueError('Incorrect datasetName: ' + datasetName + ', choose from - "train", "dev", "test" ') 
    if (os.path.exists(theFilePath)):
        with open(theFilePath, "rb") as fp:
            stanfordTags = pickle.load(fp)
            return stanfordTags
    
    else :
        #Need to create taggings!
        taggedSentsList = []
        for sents in theSents:
            tokenisedSents = [word_tokenize(sent) for sent in sents]
            classifiedSents = sTagger.tag_sents(tokenisedSents)
            taggedSentsList.append(classifiedSents)
        #And save them
        with open(theFilePath, "wb") as fp: 
            pickle.dump(taggedSentsList, fp)
        return taggedSentsList

In [5]:
tagged_train_set = getStanfordTagging('train')
tagged_dev_set = getStanfordTagging('dev')
tagged_test_set = getStanfordTagging('test')

In [ ]:
tagged_rapid_set = tagged_train_set[:rapid_size]

In [257]:
# Shim function for later clean

# Thanks for this list to save me typing it : http://stackoverflow.com/questions/493174/is-there-a-way-to-convert-number-words-to-integers\n",
numInWords = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"
       , "hundred", "thousand", "million", "billion", "trillion"]

punctuation = ['.',',',';',':','`', "'"]

def isPunctuation(word):
    return word in punctuation
def isCapitalised (word):
    if len(word) == 0:
        return False
    return word[0].isupper()

# Obtained from training data
postUnits = [u'%', u'century', u'years', u'percent', u'years ago', u'days', u'months', u'km', u'hours', u'times', u'inches', u'\xb0C', u'minutes', u'acres', u'\xb0F', u'weeks', u'people', u'sq mi', u'mi', u'ft', u'feet', u'metres', u'mm', u'square miles', u'miles', u'pm', u'per cent', u'year', u'copies', u'yuan', u'men', u'square feet', u'third', u'kilometres', u'nm', u'tonnes', u'species', u'decades', u'barrels', u'tons', u'largest', u'centuries', u'km2']
preUnits = [u'$',u'around', u'late', u'early', u'nearly', u'since', u'approximately', u'number']

# Returns true if the word represents a number\n",
def isNumber(word):
    pattern = ".?(\\d)+((,|.)(\\d)+)*"
    if re.match(pattern,word) :
        return True
    if word.lower() in numInWords:
        return True
    return False

def isStopWord(word):
    return word.lower() in stop_words

In [232]:
# Shim function for later clean

def refine_word_tags(taggedWordList):
    newWordTags = []
    for (word, tag) in taggedWordList:
        if (tag == 'ORGANIZATION'):
            tag = 'O'
        if (tag == 'O'):
            #Might be a number
            if isNumber(word):
                tag = 'NUMBER'
            elif isCapitalised(word):
                tag = 'OTHERCAP'
            elif word in preUnits:
                tag = 'PRENUM'
            elif word in postUnits:
                tag = 'POSTNUM'
            elif isStopWord(word):
                tag = 'STOPWORD'
            elif isPunctuation(word):
                tag = 'PUNC'

        newWordTags.append((word, tag))
    
    newWordTags = combineTags (newWordTags)
    return newWordTags
        
def combineTags(wordTags):
    
    newTags = []
    prevWord = wordTags[0][0]
    prevTag = wordTags[0][1]
    
    for (word, tag) in wordTags[1:]:
        if tag == 'NUMBER' and prevTag == 'PRENUM':
            prevTag = 'NUMBER'
        elif prevTag == 'PRENUM':
            prevTag = 'O'
        if tag == 'POSTNUM' and prevTag == "NUMBER":
            tag = "NUMBER"
        elif tag == "POSTNUM":
            tag = "O"
        newTags.append((prevWord, prevTag))
        prevWord = word
        prevTag = tag
    newTags.append((prevWord, prevTag))
        
    newNewTags = []
    prevWord = newTags[0][0]
    prevTag = newTags[0][1]
    if (prevTag == "OTHERCAP"):
        prevTag = "O"
        
    for (word, tag) in newTags[1:]:
#         print tag, prevTag
        if tag == prevTag :
            if word == '%':
                prevWord += word
            else :
                if prevWord == '$':
                    prevWord += word
                else :
                    prevWord += ' ' + word
        else :
            newNewTags.append((prevWord, prevTag))
            prevWord = word
            prevTag = tag
            
    newNewTags.append((prevWord, prevTag))
    
    return newNewTags

## Setup functions

In [17]:
from collections import defaultdict
def getAnswerDict(qss):
    sentDicts = defaultdict(list)
    for docID in range(0, len(qss)):
        qs = qss[docID]
        for q in qs:
            answer = q["answer"]
            answerSent = (docID, q["answer_sentence"])
            sentDicts[answerSent].append(answer)
    return sentDicts
train_sentence_contained_answers = getAnswerDict(train)


In [18]:
train_sentence_contained_answers[(0,2)] # contains the answers in that sentence

[u'long playing', u'12", 10", 7"', u'rpm']

In [67]:
def getAnswerIndex(sent, answer):
    tokenised_answer = nltk.word_tokenize(answer)
    len_tokenised_answer = len(tokenised_answer)
    tokenised_sent =  nltk.word_tokenize(sent)
    highest_index = (len(tokenised_sent) - len_tokenised_answer) + 1
    
    for i in range (0, highest_index):
            sentence_fragment = tokenised_sent[i:i+len_tokenised_answer]
            if (sentence_fragment == tokenised_answer):
                return (i, i+len_tokenised_answer)
    #print "Problem, cannot find answer index"
    #print sent
    #print answer
    #print
    return (-1,-1)


In [25]:
the_sentence = trainSents[0][0]
the_answers_contained = train_sentence_contained_answers[(0,0)]

print the_sentence
print
print the_answers_contained
print
print getAnswerIndex (the_sentence, the_answers_contained[0])

A gramophone record (phonograph record in American English) or vinyl record, commonly known as a "record", is an analogue sound storage medium in the form of a flat polyvinyl chloride (previously shellac) disc with an inscribed, modulated spiral groove.

[u'analogue sound storage medium']

(24, 28)


In [33]:
def convert_candidates_to_range(candidates):
    candidatesIndexRange = []
    index = 0
    for (words, tag) in candidates:
        tokenisedWords = nltk.word_tokenize(words)
        nextIndex = index + len(tokenisedWords)
        candidatesIndexRange.append((index,nextIndex ))
        index = nextIndex
    return candidatesIndexRange

In [32]:
example_candidates = refine_word_tags(tagged_train_set[0][0])

convert_candidates_to_range(example_candidates)[0:5]

[(0, 6), (6, 7), (7, 9), (9, 10), (10, 11)]

In [193]:
# Get the taggings of the (possibly overlapping) candidates list

def get_taggings_in_range((ans_start,ans_end),candidates):
    candidate_ranges = convert_candidates_to_range(candidates)
    
    current_considered_candidate_id = -1
    candidates_containing_answer = []
    
    for (cand_start, cand_end) in candidate_ranges:
        current_considered_candidate_id += 1
        if ans_start == cand_start and ans_end == cand_end :
            #candidates and answer the same
            candidates_containing_answer.append(current_considered_candidate_id)
            break
        if cand_start <= ans_start and cand_end >= ans_end :
            #candidate contains the answer, but has extra words
            candidates_containing_answer.append(current_considered_candidate_id)
            break

        if cand_start >= ans_start or cand_end > ans_start:
            if (cand_end <= ans_end):
                candidates_containing_answer.append(current_considered_candidate_id)
            else :
                # we have finished, as 
                if (cand_start < ans_end ):
                    candidates_containing_answer.append(current_considered_candidate_id)
                break
    return candidates_containing_answer
    

In [195]:
example_candidates = refine_word_tags(tagged_train_set[0][1])
example_answer_range = (3,6)
print example_candidates
print
print get_taggings_in_range(example_answer_range,example_candidates)

[(u'The groove usually starts near', 'O'), (u'the', 'STOPWORD'), (u'periphery', u'O'), (u'and', 'STOPWORD'), (u'ends near', u'O'), (u'the', 'STOPWORD'), (u'center', u'O'), (u'of the', 'STOPWORD'), (u'disc', u'O'), (u'.', 'PUNC')]

[0, 1]


## Analysis

In [235]:
def get_NER_tag_analysis() :
    correct_taggings = []
    incorrect_taggings = []
    evil_ones = []
    for i in range(0, len(trainSents)):
        sent_set = trainSents[i]
        for j in range(0, len(sent_set)):
            sent = sent_set[j]
            answers_in_sent = train_sentence_contained_answers[(i,j)]
            candidates = refine_word_tags(tagged_train_set[i][j])
            for answer in answers_in_sent:
                (ans_start, ans_end) = getAnswerIndex(sent, answer)
                if (ans_start, ans_end) == (-1,-1) :
                    #question = train[i][j]["question"]
                    evil_ones.append((i,j, answer))
                    continue
                   
                candidate_ids = get_taggings_in_range((ans_start, ans_end),candidates)
                if len(candidate_ids) == 1:
                    possible_correct_candidate = candidates[candidate_ids[0]]
                    if possible_correct_candidate[0] == answer :
                        # Correct tagging!
                        correct_taggings.append((possible_correct_candidate, i, j))
                    else :
                        incorrect_taggings.append(([possible_correct_candidate], answer, i, j))
                else :
                    incorrect_candidates = ([candidates[index] for index in candidate_ids],answer, i, j)
                    incorrect_taggings.append(incorrect_candidates)
        if (i == 10):
            break
    return (correct_taggings, incorrect_taggings, evil_ones)

In [236]:
(correct_taggings, incorrect_taggings, evil_ones) = get_NER_tag_analysis()

In [237]:
print len(evil_ones)

22


In [238]:
from collections import Counter
correctCounter = Counter()
for correct_tags in correct_taggings:
    correctCounter[correct_tags[0][1]] += 1
    

In [239]:
correctCounter

Counter({u'LOCATION': 86,
         'NUMBER': 317,
         u'O': 154,
         'OTHERCAP': 200,
         u'PERSON': 141,
         'STOPWORD': 1})

In [240]:
from collections import Counter
from collections import defaultdict
wrongCounter = Counter()
wrongList = defaultdict(list)

for wrong_tags in incorrect_taggings:
    tagList = []
    for (words, tag) in wrong_tags[0]:
        tagList.append(tag)
    wrongCounter[tuple(tagList)] += 1
    wrongList[tuple(tagList)].append(wrong_tags)

In [241]:
wrongCounter.most_common(10)

[((u'O',), 148),
 (('O', 'STOPWORD', u'O'), 89),
 (('OTHERCAP', 'STOPWORD', 'OTHERCAP'), 49),
 (('NUMBER',), 33),
 (('STOPWORD', u'O'), 33),
 (('O', 'OTHERCAP'), 29),
 (('NUMBER', u'O'), 26),
 (('OTHERCAP', 'NUMBER', 'PUNC', 'NUMBER'), 26),
 (('OTHERCAP', u'O'), 23),
 ((u'O', 'STOPWORD', u'O', 'STOPWORD', u'O'), 23)]

In [250]:
wrongList[('O', 'STOPWORD', u'O')][0][0][1][0]

u'the'

In [252]:
testCounter = Counter()
for tuplesy in wrongList[('O', 'STOPWORD', u'O')]:
    testCounter[tuplesy[0][1][0]] += 1

In [253]:
testCounter.most_common(5)

[(u'and', 26), (u'of', 11), (u'the', 10), (u'of the', 5), (u'or', 5)]

# TO CHANGE

## Evaluation on Dev sent for all:


In [255]:
def evaluateNERonDev():
    correct = []
    wrong = []
    
    for i in range(0, len(dev)):
        qs = dev[i]
        for j in range(0, len(qs)):
            q = qs[j]
            idSent  = q["answer_sentence"]
            sent = devSents[i][idSent]
            answer = q["answer"]
            possAnswers = refine_word_tags(tagged_dev_set[i][idSent])
            inThere = False
            for possAnswer in possAnswers:
                if possAnswer[0] == answer:
                    inThere = True
                    break
            if inThere:
                correct.append((i, j, idSent, possAnswers))
            else :
                wrong.append((i, j, idSent, possAnswers))
    return (correct, wrong)


(correct,wrong) = evaluateNERonDev()

In [256]:
print (len(correct) + 0.0)/ (len(correct) + len(wrong))


0.469927921541


In [ ]:
# Accuracy - 0.456693843791 BASE

# Fix dollar space + % space! - 0.470873212809
# Add ` and '  - 0.470991374217 loool






# POS tag on others!


In [252]:
# Shim function for later clean

def evaluate_ner(name, data):
    
    question_set = data[name]["question_set"]
    b_output_answer_set = data[name]["b_output_answer_set"]
    
    correct = []
    wrong = []
    
    for result_b in b_output_answer_set:
        
        answer = question_set[result_b["set_index"]][result_b["question_index"]]["answer"]
        
        possible_candidates = result_b["candidates"]
        
        answer_exists_in_candidates = False
        
        for candidate in possible_candidates:
            
            candidate_string = candidate[0]
            
            if candidate_string == answer:
                
                answer_exists_in_candidates = True
                
                break
        
        if answer_exists_in_candidates:
            correct.append(result_b)
        else :
            wrong.append(result_b)
            
    return (correct, wrong)

In [253]:
def process_ner(name, data, stats=False):
    print "Processing ner: ", name
    process_part_b(name, data)
    if stats:
        process_generic(name, data, "ner", evaluate_ner)
        
        correct_ner = len(data[name]["ner_correct"])
        correct_ret = len(data[name]["retrieval_correct"])
        
        avg = correct_ner * 1.0 / correct_ret
        
        print "ner".capitalize() + " Correct Average of Previous %: ", avg
    print

In [254]:
process_ner("rapid", DATA, True)

Processing ner:  rapid

Part B Output: 
[   {   'candidates': [   (u'They', 'O'),
                          (u'had a', 'STOPWORD'),
                          (u'playing time', u'O'),
                          (u'of', 'STOPWORD'),
                          (u'eight minutes', 'NUMBER'),
                          (u'.', 'PUNC')],
        'question_index': 0,
        'sentence_index': 149,
        'set_index': 0}]

Ner Correct:  67
Ner Wrong:  337
Ner Total:  404
Ner Overall Average %:  0.165841584158
Ner Correct Average of Previous %:  0.429487179487



In [271]:
process_ner("train", DATA, True)

Processing ner:  train

Part B Output: 
[   {   'candidates': [   (u'They', 'O'),
                          (u'had a', 'STOPWORD'),
                          (u'playing time', u'O'),
                          (u'of', 'STOPWORD'),
                          (u'eight minutes', 'NUMBER'),
                          (u'.', 'PUNC')],
        'question_index': 0,
        'sentence_index': 149,
        'set_index': 0}]

Ner Correct:  20067
Ner Wrong:  50092
Ner Total:  70159
Ner Overall Average %:  0.286021750595
Ner Correct Average of Previous %:  0.459419858513



In [255]:
process_ner("dev", DATA, True)

Processing ner:  dev

Part B Output: 
[   {   'candidates': [   (u'Infrared', 'O'),
                          (u'is', 'STOPWORD'),
                          (u'used', u'O'),
                          (u'in', 'STOPWORD'),
                          (u'night vision equipment', u'O'),
                          (u'when there is', 'STOPWORD'),
                          (u'insufficient visible light', u'O'),
                          (u'to', 'STOPWORD'),
                          (u'see', u'O'),
                          (u'.', 'PUNC')],
        'question_index': 0,
        'sentence_index': 71,
        'set_index': 0}]

Ner Correct:  2415
Ner Wrong:  6048
Ner Total:  8463
Ner Overall Average %:  0.285359801489
Ner Correct Average of Previous %:  0.477272727273



In [276]:
process_ner("test", DATA, False)

Processing ner:  test

Part B Output: 
[   {   'candidates': [   (u'a', 'STOPWORD'),
                          (u'forgotten theatre', u'O'),
                          (u'of the', 'STOPWORD'),
                          (u'Crimean War', 'OTHERCAP'),
                          (u'.', 'PUNC')],
        'question_index': 0,
        'sentence_index': 283,
        'set_index': 0}]




In [257]:
# Shim function for later clean

def getQuestionType(question):
    if 'Who' in question:
        return "PERSON"
    if 'where' in question:
        return "LOCATION"
    if 'How many' in question:
        return "NUMBER"
    if 'How much' in question:
        return "NUMBER"
    if 'When' in question:
        return "NUMBER"
    if 'what year' in question:
        return "NUMBER"
    else:
        return "O"

In [258]:
# First, answers whose content words all appear in the question should be ranked lowest.

def first_filter(question, answer_entities):
   
    ranked_list = []
    
    question = set(pre_process_tf_idf(question))
    
#     print question
#     print
    
    for entity in answer_entities:

        raw_span = entity[0]
        span_tag = entity[1]
        
        set_span = set(pre_process_tf_idf(raw_span))
        
        if span_tag != "O" and span_tag != "STOPWORD" and span_tag !="PUNC":
            
            if set_span.issubset(question):
                
                ranked_list.append([entity, 1])
#                 print "IN", raw_span, span_tag, set_span, question
                
            else:
                
                ranked_list.append([entity, 2])
#                 print "OUT", raw_span, span_tag, set_span, question
    
    return sorted(ranked_list, key=lambda x: x[1], reverse=True)

In [259]:
# First, answers whose content words all appear in the question should be ranked lowest.

def first_filter_object(question, answer_entities):
   
    ranked_list = []
    
    question = set(pre_process_tf_idf(question))
    
#     print question
#     print
    
    for entity in answer_entities:

        raw_span = entity[0]
        span_tag = entity[1]
        
        set_span = set(pre_process_tf_idf(raw_span))
        
        if span_tag != "STOPWORD" and span_tag !="PUNC": #span_tag != "O" and
            
            if span_tag == "O":
                
                if len(set_span) > 2:
                    ranked_list.append([entity, 0])
            
            elif set_span.issubset(question):
                
                ranked_list.append([entity, 1])
#                 print "IN", raw_span, span_tag, set_span, question
                
            else:
                
                ranked_list.append([entity, 2])
#                 print "OUT", raw_span, span_tag, set_span, question
    
    return sorted(ranked_list, key=lambda x: x[1], reverse=True)

In [283]:
# First, answers whose content words all appear in the question should be ranked lowest.

def first_filter_object_stop(question, answer_entities):
   
    ranked_list = []
    
    question = set(pre_process_tf_idf(question))
    
#     print question
#     print
    
    for entity in answer_entities:

        raw_span = entity[0]
        span_tag = entity[1]
        
        set_span = set(pre_process_tf_idf(raw_span))
        
        if span_tag !="PUNC": #span_tag != "O" and
            
            if span_tag == "O" or span_tag == "STOPWORD":
                
                if len(set_span) > 2:
                    
                    ranked_list.append([entity, 0])
            
            elif set_span.issubset(question):
                
                ranked_list.append([entity, 1])
#                 print "IN", raw_span, span_tag, set_span, question
                
            else:
                
                ranked_list.append([entity, 2])
#                 print "OUT", raw_span, span_tag, set_span, question
    
    return sorted(ranked_list, key=lambda x: x[1], reverse=True)

In [284]:
# Second, answers which match the question type should be ranked higher than those that don't; for this, you
# should build a simple rule-based question type classifier based on key words (e.g. questions which contain "who" are
# people).

# First, answers whose content words all appear in the question should be ranked lowest.

def second_filter(question, ranked_list):
   
    question_type = getQuestionType(question)
#     print question_type
    
    for index, answer in enumerate(ranked_list):
        
        entity_tag = answer[0][1]
        
        if entity_tag == question_type:
#             print "MATCH", answer[0], question_type, question
            ranked_list[index].append(2)
#             ranked_list[index][1] += 1
        else:
            ranked_list[index].append(1)
#             ranked_list[index][1] -= 1
            
    return ranked_list

In [285]:
def pre_process_open_class(line):
    tokenized_sentence = word_tokenizer.tokenize(line.lower())
    lemmatized_sentence = [lemmatize(token) for token in tokenized_sentence]
    filtered_sentence = [token for token in lemmatized_sentence if token not in filter_tokens]
    tagged_sent = nltk.pos_tag(lemmatized_sentence)
    final = []
    for word, tag in tagged_sent:
        if "V" in tag or "NN" in tag:
#             final.append((word,tag))
            final.append(word)
            
#     print "RESULT: ", final
    return final

In [286]:
# Third, among entities of the same type, the prefered entity should be the one which is closer in the sentence to a
# closed-class word from the question.

def third_filter(question, possAnswers, ranked_list):
    
    question = pre_process_open_class(question)

    answer_sent = " ".join([x[0] for x in possAnswers])
    answer_sent = pre_process_tf_idf(answer_sent)
    raw_answer_sent = " ".join(answer_sent)
    
#     print "QUESTION: "
#     pp.pprint(question)
#     print "ANSWER: "
#     pp.pprint(answer_sent)
#     pp.pprint(raw_answer_sent)
    
    for index, answer in enumerate(ranked_list):

        span_tag = answer[0][1]
        raw_span = answer[0][0]

        proc_span = pre_process_tf_idf(raw_span)

        raw_proc_span = " ".join(proc_span)
        new_raw_proc_span = "-".join(proc_span)

        raw_answer_sent = raw_answer_sent.replace(raw_proc_span, new_raw_proc_span)
    
    answer_sent = raw_answer_sent.split(" ")
    
    avg_dict = defaultdict(float)
    
    for open_class in question:
        
        if open_class in answer_sent:
            
            open_class_locations = [i for i, x in enumerate(answer_sent) if x == open_class]
            
#             print "OPEN CLASS: ", repr(open_class)

            for index, answer in enumerate(ranked_list):

                span_tag = answer[0][1]
                raw_span = answer[0][0]

                proc_span = pre_process_tf_idf(raw_span)
                
                raw_proc_span = " ".join(proc_span)
                new_raw_proc_span = "-".join(proc_span)
                
                proc_span_locations = [i for i, x in enumerate(answer_sent) if x == new_raw_proc_span]
                
                min_dist = len(answer_sent)
                min_dist_ind = (None, None)
                
                for loc1 in proc_span_locations:
                    
                    for loc2 in open_class_locations:
                        
                        dist = abs(loc1 - loc2)
                        
                        if dist < min_dist:
                            
                            min_dist = dist
                            min_dist_ind = (loc1, loc2)
                
#                 print "PROC: ", proc_span_locations
#                 print "OPEN CLASS: ", open_class_locations                
                scale = (len(answer_sent) - min_dist) * 1.0 / len(answer_sent)
#                 print "JOINT: ", min_dist_ind, scale
                avg_dict[index] += scale
#                 ranked_list[index][1] *= scale
    
    for key, value in avg_dict.iteritems():
        ranked_list[key].append(value / len(question))

    return ranked_list

In [287]:
def reduce_rank(ranking_list):
    
    new_ranking = []
    
    for rank in ranking_list:
        
        new_rank = ( rank[1] + rank[2] )
        
        if len(rank) == 4:
             new_rank *= rank[3]
        
        new_ranking.append([rank[0], new_rank])
        
    return sorted(new_ranking, key=lambda x: x[1], reverse=True)

In [288]:
import random


In [298]:
def generate_part_c_output(name, data):
    
    part_c_output = []

    question_set = data[name]["question_set"]
    document_set = data[name]["document_set"]
        
    b_output_answer_set = data[name]["b_output_answer_set"]
    
    for result_b in b_output_answer_set:
        
        question = question_set[result_b["set_index"]][result_b["question_index"]]["question"]
        
        first_pass = first_filter(question, result_b["candidates"])
        
        second_pass = second_filter(question, first_pass)
        
        third_pass = third_filter(question, result_b["candidates"], second_pass)
        
        fourth_pass = reduce_rank(third_pass)
                
#         pp.pprint(third_pass)

        predicted_answer = None

        if len(fourth_pass) > 0:

            top_answer = fourth_pass[0]
    #         pp.pprint(top_answer)        
            predicted_answer = top_answer[0][0]
        
        else:
            
            first_pass = first_filter_object(question, result_b["candidates"])

            second_pass = second_filter(question, first_pass)

            third_pass = third_filter(question, result_b["candidates"], second_pass)

            fourth_pass = reduce_rank(third_pass)   
            
            if len(fourth_pass) > 0:

                top_answer = fourth_pass[0]
        #         pp.pprint(top_answer)        
                predicted_answer = top_answer[0][0]
            
            else:
                
                first_pass = first_filter_object_stop(question, result_b["candidates"])

                second_pass = second_filter(question, first_pass)

                third_pass = third_filter(question, result_b["candidates"], second_pass)

                fourth_pass = reduce_rank(third_pass)   

                if len(fourth_pass) > 0:

                    top_answer = fourth_pass[0]
            #         pp.pprint(top_answer)        
                    predicted_answer = top_answer[0][0]                
                
                else:

        #         pp.pprint(top_answer)        
                    predicted_answer = random.choice(result_b["candidates"])[0]
            
        
        result_c = {
            "set_index"  : result_b["set_index"],
            "question_index" : result_b["question_index"],
            "sentence_index" : result_b["sentence_index"],
            "candidates": result_b["candidates"],
            "ranked_answers": fourth_pass,
            "predicted_answer" : predicted_answer
        }
        
        part_c_output.append(result_c)        

    return part_c_output

In [299]:
def process_part_c(name, data):
    
    data[name]["c_output_answer_set"] = generate_part_c_output(name, data)
    
    print
    print "Part C Output: "
    pp.pprint(data[name]["c_output_answer_set"][:rapid_size])
    print    

In [300]:
# For each question, evaluate if the answer is present as an entity

def evaluate_rank(name, data):
    
    question_set = data[name]["question_set"]
    document_set = data[name]["document_set"]
    
    correct = []
    wrong = []
    
    c_output_answer_set = data[name]["c_output_answer_set"]
    
    for result_c in c_output_answer_set:
        
        question = question_set[result_c["set_index"]][result_c["question_index"]]["question"]
        answer =  question_set[result_c["set_index"]][result_c["question_index"]]["answer"]
        
        predicted_answer = result_c["predicted_answer"]

        if (predicted_answer == answer):
            correct.append(result_c)
        else :
            wrong.append(result_c)
#         break
        #print correct
    return (correct, wrong)
    

In [301]:
def process_rank(name, data, stats=False):
    print "Processing rank: ", name
    process_part_c(name, data)
    if stats:
        process_generic(name, data, "rank", evaluate_rank)
        
        
        correct_rank = len(data[name]["rank_correct"])
        correct_ner = len(data[name]["ner_correct"])
        
        avg = correct_rank * 1.0 / correct_ner
        
        print "rank".capitalize() + " Correct Average of Previous %: ", avg        
        
    print

In [302]:
process_rank("rapid", DATA, True)

Processing rank:  rapid

Part C Output: 
[   {   'candidates': [   (u'They', 'O'),
                          (u'had a', 'STOPWORD'),
                          (u'playing time', u'O'),
                          (u'of', 'STOPWORD'),
                          (u'eight minutes', 'NUMBER'),
                          (u'.', 'PUNC')],
        'predicted_answer': u'eight minutes',
        'question_index': 0,
        'ranked_answers': [[(u'eight minutes', 'NUMBER'), 0.4]],
        'sentence_index': 149,
        'set_index': 0}]

Rank Correct:  36
Rank Wrong:  368
Rank Total:  404
Rank Overall Average %:  0.0891089108911
Rank Correct Average of Previous %:  0.537313432836



In [272]:
process_rank("train", DATA, True)

Processing rank:  train

Part C Output: 
[   {   'candidates': [   (u'They', 'O'),
                          (u'had a', 'STOPWORD'),
                          (u'playing time', u'O'),
                          (u'of', 'STOPWORD'),
                          (u'eight minutes', 'NUMBER'),
                          (u'.', 'PUNC')],
        'predicted_answer': u'eight minutes',
        'question_index': 0,
        'ranked_answers': [[(u'eight minutes', 'NUMBER'), 0.4]],
        'sentence_index': 149,
        'set_index': 0}]

Rank Correct:  9895
Rank Wrong:  60264
Rank Total:  70159
Rank Overall Average %:  0.141036787868
Rank Correct Average of Previous %:  0.493098121294



In [303]:
process_rank("dev", DATA, True)

Processing rank:  dev

Part C Output: 
[   {   'candidates': [   (u'Infrared', 'O'),
                          (u'is', 'STOPWORD'),
                          (u'used', u'O'),
                          (u'in', 'STOPWORD'),
                          (u'night vision equipment', u'O'),
                          (u'when there is', 'STOPWORD'),
                          (u'insufficient visible light', u'O'),
                          (u'to', 'STOPWORD'),
                          (u'see', u'O'),
                          (u'.', 'PUNC')],
        'predicted_answer': u'night vision equipment',
        'question_index': 0,
        'ranked_answers': [   [   (u'night vision equipment', u'O'),
                                  0.26666666666666666],
                              [   (u'insufficient visible light', u'O'),
                                  0.19999999999999998]],
        'sentence_index': 71,
        'set_index': 0}]

Rank Correct:  1135
Rank Wrong:  7328
Rank Total:  8463
Rank Overal

In [15]:
def log_wrong_debug(name, data):
    
    question_set = data[name]["question_set"]
    document_set = data[name]["document_set"]
    rank_wrong = data[name]["rank_wrong"]
    
    for result_wrong in rank_wrong:
        
        question = question_set[result_wrong["set_index"]][result_wrong["question_index"]]
        candidate_sentence = document_set[result_wrong["set_index"]][result_wrong["sentence_index"]]
        correct_sentence = document_set[result_wrong["set_index"]][question["answer_sentence"]]
        
        candidates = result_wrong["candidates"]
        ranked_answers = result_wrong["ranked_answers"]
        predicted_answer = result_wrong["predicted_answer"]
        
        if question["answer_sentence"] == result_wrong["sentence_index"]:
            
            print "=" * 20
            print "=" * 20
        
            print "Question: "
            print
            pp.pprint(question["question"])

            print
            print "Correct Sentence: (Part A)"
            print
            pp.pprint(correct_sentence)
            print
            print "Chosen Sentence: (Part A)"
            print
            pp.pprint(candidate_sentence)
            print

            print "Candidate Answers: (Part B)"
            print
            pp.pprint(candidates)
            print
            print "Ranked Answers: (Part C)"
            print
            pp.pprint(ranked_answers)
            print
            print "Predicted Answer: (Part C)"
            print
            pp.pprint(predicted_answer)
            print
            print "Correct Answer: (Part C)"
            print
            pp.pprint(question["answer"])     
            break

In [282]:
log_wrong_debug("rapid", DATA)

Question: 

u'What was the primary use of a phonographic disc record?'

Correct Sentence: (Part A)

u'The phonograph disc record was the primary medium used for music reproduction until late in the 20th century, replacing the phonograph cylinder record\u2013with which it had co-existed from the late 1880s through to the 1920s\u2013by the late 1920s.'

Chosen Sentence: (Part A)

u'The phonograph disc record was the primary medium used for music reproduction until late in the 20th century, replacing the phonograph cylinder record\u2013with which it had co-existed from the late 1880s through to the 1920s\u2013by the late 1920s.'

Candidate Answers: (Part B)

[   (u'The phonograph disc record', 'O'),
    (u'was the', 'STOPWORD'),
    (u'primary medium used', u'O'),
    (u'for', 'STOPWORD'),
    (u'music reproduction', u'O'),
    (u'until', 'STOPWORD'),
    (u'late', 'O'),
    (u'in the', 'STOPWORD'),
    (u'20th century', 'NUMBER'),
    (u',', 'PUNC'),
    (u'replacing', u'O'),
    (u'the'

In [304]:
process_rank("test", DATA, False)

Processing rank:  test

Part C Output: 
[   {   'candidates': [   (u'a', 'STOPWORD'),
                          (u'forgotten theatre', u'O'),
                          (u'of the', 'STOPWORD'),
                          (u'Crimean War', 'OTHERCAP'),
                          (u'.', 'PUNC')],
        'predicted_answer': u'Crimean War',
        'question_index': 0,
        'ranked_answers': [[(u'Crimean War', 'OTHERCAP'), 2]],
        'sentence_index': 283,
        'set_index': 0}]




In [278]:
def process_submit(name, data):
    
    headers = ['id', 'answer']
    
    c_output_answer_set = data[name]["c_output_answer_set"]       

    with open(name + '.submit.csv', 'w') as f:

        f_csv = csv.DictWriter(f, headers)
        f_csv.writeheader()

        for index, result_c in enumerate(c_output_answer_set):
            
            predicted_answer = result_c["predicted_answer"]
            
            if predicted_answer is not None:
                f_csv.writerows([{'id':index+1,'answer':predicted_answer.encode("utf-8")}])
            else:
                f_csv.writerows([{'id':index+1,'answer':"NONE"}])
            
#             if isinstance( answer_list[index]['answer'], int):
                
#                 f_csv.writerows([{'id':index+1,'answer':answer_list[index]['answer'][0][0]}])
                
#             else:
                
#                 f_csv.writerows([{'id':index+1,'answer':answer_list[index]['answer'][0][0].encode("utf-8")}])        
                

In [279]:
process_submit("rapid", DATA)

In [305]:
process_submit("test", DATA)